colab version

# colab set up

In [1]:
! pip install stim
! pip install -U ldpc
! pip install bposd
! pip install graph_tools
! pip install beliefmatching
! pip install parmap

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
path = "/content/drive/MyDrive/caltech surf 2025/src"
os.chdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# imports + helper functions + code set up

In [3]:
import numpy as np
import stim
import itertools
import copy
import multiprocessing as mp
import time
from bposd.css import css_code

import sys
sys.path.append('./src/')
from CCZCircuit import build_three_block_circuit_with_CCZ

In [4]:
path = "/content/drive/MyDrive/caltech surf 2025"
os.chdir(path)

In [5]:
import joblib

algebraic_toric_hx_dict_3d = joblib.load('algebraic_toric_hx_dict.pkl')
algebraic_toric_hz_dict_3d = joblib.load('algebraic_toric_hz_dict.pkl')
P_CCZ_mat_d3_toric = joblib.load('P_CCZ_mat_toric_81_3_3.pkl')
P_CCZ_mat_d5_toric = joblib.load('P_CCZ_mat_toric_375_3_5.pkl')

In [6]:
def GetLFR_PostSelected(detector_vals, logical_vals):
    # Find indices where all detector values are zero
    postselected_rows = np.all(detector_vals == 0, axis=1)
    num_postselected_shots = np.count_nonzero(postselected_rows)
    if num_postselected_shots == 0:
        print("No post-selected shots retained.")
        return None, 0
    # if any of the three observables is "True" (error), then row is a logical error
    rows_with_logical_error = np.any(logical_vals[postselected_rows, :], axis=1)
    num_rows_with_logical_error = np.count_nonzero(rows_with_logical_error)
    return num_rows_with_logical_error, num_postselected_shots

In [7]:
def run_three_block_CCZ_error_detection(eval_code,
                                        circuit_error_params,
                                        p,
                                        P_CCZ_mat_for_this_code,
                                        num_shots,
                                        num_shots_per_batch):
    ccz_circuit = build_three_block_circuit_with_CCZ(eval_code, circuit_error_params, p, 1, P_CCZ_mat_for_this_code)
    sampler = ccz_circuit.compile_detector_sampler()
    num_batches, leftover = num_shots//num_shots_per_batch, num_shots%num_shots_per_batch
    print(f"num_batches {num_batches}, num_shots_per_batch {num_shots_per_batch}, leftover {leftover}")
    num_obs = eval_code.lx.shape[0]*3 # 3k = 9 observables
    postselected_shots = 0
    total_failure_count = 0
    total_shots = 0
    for batch_i in range(num_batches+1):
        curr_batch_shots = 0
        if batch_i == num_batches:
            if leftover == 0:
                break
            batch_shots = sampler.sample(leftover, prepend_observables=True)
            curr_batch_shots = leftover
        else:
            batch_shots = sampler.sample(num_shots_per_batch, prepend_observables=True)
            curr_batch_shots = num_shots_per_batch
        batch_logical_vals = batch_shots[:, :num_obs]
        batch_detector_vals = batch_shots[:, num_obs:]
        batch_postselected_failure_count, batch_postselected_shots = GetLFR_PostSelected(batch_detector_vals, batch_logical_vals)
        total_shots += curr_batch_shots
        batch_postselected_LFR = None
        if batch_postselected_shots != 0:
            batch_postselected_LFR = batch_postselected_failure_count/batch_postselected_shots
            postselected_shots += batch_postselected_shots
            total_failure_count += batch_postselected_failure_count
        if batch_i%50 == 0:
            so_far_postselected_LFR = total_failure_count/postselected_shots
            so_far_norm_ps_LFR = 1-(1-so_far_postselected_LFR)**(1/(3*eval_code.K))
            so_far_success_rate = postselected_shots/total_shots
            print(f"batch {batch_i:4}/{num_batches} | total_shots = {total_shots:10} | p_CCZ = {p_CCZ_val:.3f} | success rate {postselected_shots:9}/{total_shots:10} = {so_far_success_rate:.3f} | unnorm postselected LFR {total_failure_count:5}/{postselected_shots:9} = {so_far_postselected_LFR}")
    print(f"FINISHED: {code_id}")
    unnormed_ps_LFR = total_failure_count/postselected_shots
    norm_ps_LFR = 1-(1-unnormed_ps_LFR)**(1/(3*eval_code.K))
    success_rate = postselected_shots/num_shots
    # print(f"num_shots = {num_shots} | p_CCZ = {p_CCZ_val:.3f} | success rate {postselected_shots}/{num_shots} = {success_rate} | unnorm PS LFR {total_failure_count}/{postselected_shots} = {postselected_LFR} | norm PS LFR {norm_ps_LFR}")
    N = postselected_shots
    unnormed_std = ((unnormed_ps_LFR)*(1-unnormed_ps_LFR)/N)**0.5
    normed_std = ((norm_ps_LFR)*(1-norm_ps_LFR)/N)**0.5
    print(f"num_shots = {num_shots} | p_CCZ = {p_CCZ_val} | success rate {postselected_shots}/{num_shots} = {postselected_shots/num_shots}")
    print(f"unnorm LFR = {unnormed_ps_LFR}")
    print(f"unnorm std = {unnormed_std}")
    print(f"norm LFR = {norm_ps_LFR}")
    print(f"norm std = {normed_std}")
    return {"unnormed_ps_LFR": unnormed_ps_LFR,
            "total_failure_count": total_failure_count,
            "postselected_shots": postselected_shots,
            "norm_ps_LFR": norm_ps_LFR,
            "success_rate": success_rate,
            "unnormed_std": unnormed_std,
            "normed_std": normed_std,
            "num_obs": num_obs,
            "ccz_circuit": ccz_circuit}

## set up 3D toric codes

In [8]:
codes_3D = {'d3_toric': None,
            'd5_toric': None}

for code_id in codes_3D:
    d = int(code_id[1])
    code_3D = css_code(algebraic_toric_hx_dict_3d[d], algebraic_toric_hz_dict_3d[d])
    code_3D.D = d
    print(f"eval_code: {code_3D.code_params}")
    codes_3D[code_id] = code_3D

for code in codes_3D.values():
    code.hx = code.hx.toarray()
    code.hz = code.hz.toarray()
    code.lx = code.lx.toarray()
    code.lz = code.lz.toarray()

eval_code: (nan,nan)-[[81,3,3]]
eval_code: (nan,nan)-[[375,3,5]]


# d3 toric with CCZ

In [9]:
code_id = 'd3_toric'
P_CCZ_mat_for_this_code = P_CCZ_mat_d3_toric
eval_code = codes_3D[code_id]
p = 0.001
p_CCZ_val = 0.001
num_shots = 500_000_000 # N = 100000000 so std = 0.1 e-06

print(f"CCZ via weight-5 Z errors", code_id, eval_code.code_params)
circuit_error_params = {"p_i": 0, "p_state_p": 1, "p_m": 1, "p_CX":1, "p_idling_gate": 0, "p_CCZ": p_CCZ_val}
num_shots_per_batch = 1_000_000

toric_d3_results = run_three_block_CCZ_error_detection(eval_code, circuit_error_params, p, P_CCZ_mat_for_this_code, num_shots, num_shots_per_batch)

CCZ via weight-5 Z errors d3_toric (nan,nan)-[[81,3,3]]
num_batches 500, num_shots_per_batch 1000000, leftover 0
batch    0/500 | total_shots =    1000000 | p_CCZ = 0.001 | success rate    349771/   1000000 = 0.350 | unnorm postselected LFR     2/   349771 = 5.7180269376249036e-06
batch   50/500 | total_shots =   51000000 | p_CCZ = 0.001 | success rate  17802343/  51000000 = 0.349 | unnorm postselected LFR    38/ 17802343 = 2.134550491471825e-06
batch  100/500 | total_shots =  101000000 | p_CCZ = 0.001 | success rate  35252984/ 101000000 = 0.349 | unnorm postselected LFR    71/ 35252984 = 2.014013905886662e-06
batch  150/500 | total_shots =  151000000 | p_CCZ = 0.001 | success rate  52703141/ 151000000 = 0.349 | unnorm postselected LFR   106/ 52703141 = 2.011265324774476e-06
batch  200/500 | total_shots =  201000000 | p_CCZ = 0.001 | success rate  70158118/ 201000000 = 0.349 | unnorm postselected LFR   139/ 70158118 = 1.981239006439711e-06
batch  250/500 | total_shots =  251000000 | p_

In [10]:
toric_d3_results["ccz_circuit"]

stim.Circuit('''
    RX 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456 457 458
    R 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 

# d5 toric with CCZ

In [11]:
code_id = 'd5_toric'
P_CCZ_mat_for_this_code = P_CCZ_mat_d5_toric
eval_code = codes_3D[code_id]
p = 0.001
p_CCZ_val = 0.001
num_shots = 10_000_000 # just enough to get success rate

print(f"CCZ via weight-5 Z errors", code_id, eval_code.code_params)
circuit_error_params = {"p_i": 0, "p_state_p": 1, "p_m": 1, "p_CX":1, "p_idling_gate": 0, "p_CCZ": p_CCZ_val}
num_shots_per_batch = 1_000_000

toric_d5_results = run_three_block_CCZ_error_detection(eval_code, circuit_error_params, p, P_CCZ_mat_for_this_code, num_shots, num_shots_per_batch)

CCZ via weight-5 Z errors d5_toric (nan,nan)-[[375,3,5]]
num_batches 1, num_shots_per_batch 1000000, leftover 0
batch    0/1 | total_shots =    1000000 | p_CCZ = 0.001 | success rate      7741/   1000000 = 0.008 | unnorm postselected LFR     0/     7741 = 0.0
FINISHED: d5_toric
num_shots = 1000000 | p_CCZ = 0.001 | success rate 7741/1000000 = 0.007741
unnorm LFR = 0.0
unnorm std = 0.0
norm LFR = 0.0
norm std = 0.0


In [12]:
toric_d5_results["ccz_circuit"]

stim.Circuit('''
    RX 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 27

# set up 3D blp codes

In [13]:
blp_hx_dict_3d = joblib.load('hx_dict_3d.pkl')
blp_hz_dict_3d = joblib.load('hz_dict_3d.pkl')
P_CCZ_mat_d3_blp = joblib.load('P_CCZ_mat_27_3_3.pkl')
P_CCZ_mat_d5_blp = np.load('P_CCZ_mat_81_3_5.npy')

In [14]:
for code_id in ['d3_blp', 'd5_blp']:
    d = int(code_id[1])
    code_3D = css_code(blp_hx_dict_3d[d], blp_hz_dict_3d[d])
    code_3D.D = d
    print(f"eval_code: {code_3D.code_params}")
    codes_3D[code_id] = code_3D

for code in codes_3D.values():
    if type(code.hx) is not np.ndarray:
        code.hx = code.hx.toarray()
        code.hz = code.hz.toarray()
        code.lx = code.lx.toarray()
        code.lz = code.lz.toarray()

eval_code: (nan,nan)-[[27,3,3]]
eval_code: (nan,nan)-[[81,3,5]]


# d3 BLP with CCZ

In [15]:
code_id = 'd3_blp'
P_CCZ_mat_for_this_code = P_CCZ_mat_d3_blp
eval_code = codes_3D[code_id]
p = 0.001
p_CCZ_val = 0.001
num_shots = 1_000_000_000

print(f"CCZ via weight-5 Z errors", code_id, eval_code.code_params)
circuit_error_params = {"p_i": 0, "p_state_p": 1, "p_m": 1, "p_CX":1, "p_idling_gate": 0, "p_CCZ": p_CCZ_val}
num_shots_per_batch = 1_000_000

blp_d3_results = run_three_block_CCZ_error_detection(eval_code, circuit_error_params, p, P_CCZ_mat_for_this_code, num_shots, num_shots_per_batch)

CCZ via weight-5 Z errors d3_blp (nan,nan)-[[27,3,3]]
num_batches 1000, num_shots_per_batch 1000000, leftover 0
batch    0/1000 | total_shots =    1000000 | p_CCZ = 0.001 | success rate    703682/   1000000 = 0.704 | unnorm postselected LFR     0/   703682 = 0.0
batch   50/1000 | total_shots =   51000000 | p_CCZ = 0.001 | success rate  35905196/  51000000 = 0.704 | unnorm postselected LFR    53/ 35905196 = 1.4761094745172815e-06
batch  100/1000 | total_shots =  101000000 | p_CCZ = 0.001 | success rate  71108433/ 101000000 = 0.704 | unnorm postselected LFR   109/ 71108433 = 1.5328702293299025e-06
batch  150/1000 | total_shots =  151000000 | p_CCZ = 0.001 | success rate 106312676/ 151000000 = 0.704 | unnorm postselected LFR   172/106312676 = 1.6178691617168963e-06
batch  200/1000 | total_shots =  201000000 | p_CCZ = 0.001 | success rate 141519715/ 201000000 = 0.704 | unnorm postselected LFR   246/141519715 = 1.738273709779588e-06
batch  250/1000 | total_shots =  251000000 | p_CCZ = 0.001

In [16]:
blp_d3_results["ccz_circuit"]

stim.Circuit('''
    RX 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152
    R 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
    DEPOLARIZE1(0.001) 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
    DEPOLARIZE

# d5 BLP w CCZ

(note: this is repeated total x5 in separate notebooks, run in parallel. results are pooled)

In [1]:
code_id = 'd5_blp'
P_CCZ_mat_for_this_code = P_CCZ_mat_d5_blp
eval_code = codes_3D[code_id]
p = 0.001
p_CCZ_val = 0.001
num_shots = 50_000_000_000

print(f"CCZ via weight-5 Z errors", code_id, eval_code.code_params)
circuit_error_params = {"p_i": 0, "p_state_p": 1, "p_m": 1, "p_CX":1, "p_idling_gate": 0, "p_CCZ": p_CCZ_val}
num_shots_per_batch = 1_000_000

blp_d5_results = run_three_block_CCZ_error_detection(eval_code, circuit_error_params, p, P_CCZ_mat_for_this_code, num_shots, num_shots_per_batch)

NameError: name 'P_CCZ_mat_d5_blp' is not defined

In [ ]:
blp_d5_results["ccz_circuit"]